<a id='CPI PREDICTION PROJECT'></a>
<font size="+20" color='#780404'><b> CPI PREDICTION PROJECT</b></font>  

<a id='CPI PREDICTION PROJECT'></a>
<font size="+5" color='#780404'><b> Importing Libraries</b></font>  

In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
plt.style.use('ggplot') #Applying style to graphs
import termcolor #for customization of text
from plotly.subplots import make_subplots
import plotly.graph_objects as go
plt.style.use('ggplot') #Applying style to graphs
import termcolor #for customization of text
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix, roc_auc_score

<a id='feature-correlation'></a>
<font size="+6" color='#780404'><b> Loading Data</b></font>  

In [2]:
# Set display options to show all columns
pd.set_option('display.max_columns', None)

In [3]:
cpi_hist = pd.read_csv('CPI_HistoryMar_23.csv')
vehicles = pd.read_csv('Naamsa_Vehicle_Sales.csv')

In [4]:
cpi_hist.head()

,Month,Category,Value,Percentage Change (From Prior Month)
0,31-03-2023,Headline_CPI,109.0,1.0
1,31-03-2023,Food and non-alcoholic beverages,116.7,1.0
2,31-03-2023,Alcoholic beverages and tobacco,109.2,2.2
3,31-03-2023,Clothing and footwear,103.4,-0.1
4,31-03-2023,Housing and utilities,104.5,0.4


<a id='feature-correlation'></a>
<font size="+4" color='#780404'><b> Data Preprocessing </b></font>  

In [5]:
cpi_hist.shape

(195, 4)

In [6]:
cpi_hist.isna().sum()

Month                                   0
Category                                0
Value                                   0
Percentage Change (From Prior Month)    0
dtype: int64

In [7]:
cpi_hist.duplicated().sum()

0

In [8]:
cpi_hist['Category'].unique()

array(['Headline_CPI', 'Food and non-alcoholic beverages',
       'Alcoholic beverages and tobacco', 'Clothing and footwear',
       'Housing and utilities', 'Household contents and services',
       'Health', 'Transport', 'Communication', 'Recreation and culture',
       'Education', 'Restaurants and hotels ',
       'Miscellaneous goods and services'], dtype=object)

In [9]:
cpi_hist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 4 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Month                                 195 non-null    object 
 1   Category                              195 non-null    object 
 2   Value                                 195 non-null    float64
 3   Percentage Change (From Prior Month)  195 non-null    float64
dtypes: float64(2), object(2)
memory usage: 6.2+ KB


In [10]:
cpi_hist['Month'] = pd.to_datetime(cpi_hist['Month']) #converting the month column to a datetime format

/var/folders/mc/wtr2th093fscq0l5wyyg1yg00000gq/T/ipykernel_38473/587887480.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  cpi_hist['Month'] = pd.to_datetime(cpi_hist['Month']) #converting the month column to a datetime format


In [11]:
cpi_pivot = cpi_hist.pivot(index = 'Month', columns = 'Category', values = 'Value').reset_index()#changing from a long format to a wide format,
cpi_pivot = cpi_pivot.sort_values("Month").reset_index(drop=True)
cpi_pivot

Category,Month,Alcoholic beverages and tobacco,Clothing and footwear,Communication,Education,Food and non-alcoholic beverages,Headline_CPI,Health,Household contents and services,Housing and utilities,Miscellaneous goods and services,Recreation and culture,Restaurants and hotels,Transport
0,2022-01-31,100.0,100.2,99.8,100.0,100.9,100.2,100.1,100.4,100.0,100.6,100.2,101.2,98.9
1,2022-02-28,100.6,100.3,99.7,100.0,101.8,100.8,103.2,100.4,100.1,101.5,100.3,101.9,100.6
2,2022-03-31,102.8,100.5,99.7,104.4,102.4,101.8,103.7,100.8,100.5,101.9,100.4,103.0,103.7
3,2022-04-30,103.6,100.7,99.6,104.4,103.1,102.4,104.0,101.1,100.5,102.8,100.7,103.3,105.1
4,2022-05-31,104.4,100.9,99.9,104.4,105.3,103.1,104.3,101.8,100.6,102.9,101.0,103.8,105.9
5,2022-06-30,104.5,101.3,100.3,104.4,106.6,104.2,104.6,103.0,101.2,103.0,101.2,104.2,110.3
6,2022-07-31,104.4,101.6,99.4,104.4,107.8,105.8,104.1,103.3,103.1,103.4,102.3,104.6,115.6
7,2022-08-31,104.7,102.1,100.1,104.4,109.7,106.0,104.2,103.7,103.2,103.5,102.4,104.3,114.5
8,2022-09-30,105.6,102.5,99.9,104.4,110.3,106.1,104.0,104.5,103.7,103.8,102.5,106.2,112.0
9,2022-10-31,106.1,102.6,99.8,104.4,111.3,106.5,104.5,105.0,103.8,104.7,102.7,107.4,111.7


In [12]:
cpi_pivot.shape

(15, 14)

## adding a new row for April

In [13]:
date_str = '2023-04-30'
date_obj = pd.to_datetime(date_str)
new_row = pd.DataFrame({'Month': [date_obj]})
cpi_pivot = pd.concat([cpi_pivot, new_row]).reset_index(drop=True)
cpi_pivot

,Month,Alcoholic beverages and tobacco,Clothing and footwear,Communication,Education,Food and non-alcoholic beverages,Headline_CPI,Health,Household contents and services,Housing and utilities,Miscellaneous goods and services,Recreation and culture,Restaurants and hotels,Transport
0,2022-01-31,100.0,100.2,99.8,100.0,100.9,100.2,100.1,100.4,100.0,100.6,100.2,101.2,98.9
1,2022-02-28,100.6,100.3,99.7,100.0,101.8,100.8,103.2,100.4,100.1,101.5,100.3,101.9,100.6
2,2022-03-31,102.8,100.5,99.7,104.4,102.4,101.8,103.7,100.8,100.5,101.9,100.4,103.0,103.7
3,2022-04-30,103.6,100.7,99.6,104.4,103.1,102.4,104.0,101.1,100.5,102.8,100.7,103.3,105.1
4,2022-05-31,104.4,100.9,99.9,104.4,105.3,103.1,104.3,101.8,100.6,102.9,101.0,103.8,105.9
5,2022-06-30,104.5,101.3,100.3,104.4,106.6,104.2,104.6,103.0,101.2,103.0,101.2,104.2,110.3
6,2022-07-31,104.4,101.6,99.4,104.4,107.8,105.8,104.1,103.3,103.1,103.4,102.3,104.6,115.6
7,2022-08-31,104.7,102.1,100.1,104.4,109.7,106.0,104.2,103.7,103.2,103.5,102.4,104.3,114.5
8,2022-09-30,105.6,102.5,99.9,104.4,110.3,106.1,104.0,104.5,103.7,103.8,102.5,106.2,112.0
9,2022-10-31,106.1,102.6,99.8,104.4,111.3,106.5,104.5,105.0,103.8,104.7,102.7,107.4,111.7


- NB: The new row added for April has null values. We will demonstrate how we deal with these later.

In [14]:
cpi_pivot.shape

(16, 14)

In [15]:
import datetime
cpi_pivot['year_month'] = pd.to_datetime(cpi_pivot['Month'], format='%Y-%b').dt.strftime('%Y-%m')
start_date = datetime.datetime.strptime("2020-12-31", "%Y-%m-%d")
end_date = datetime.datetime.strptime("2023-03-31", "%Y-%m-%d")

# difference between each date. M means one month end
D = 'M'

date_list = pd.date_range(start_date, end_date, freq=D)[::-1]
vehicles['Date'] = date_list
vehicles['Date'] = pd.to_datetime(vehicles['Date'], format='%Y-%b-%d')
vehicles['year_month'] = pd.to_datetime(vehicles['Date'], format='%Y-%b').dt.strftime('%Y-%m')

cpi_pivot = cpi_pivot.merge(vehicles[['year_month', 'Total_Local Sales', 'Total_Export_Sales']], on='year_month', how='left')


<a id='feature-correlation'></a>
<font size="+2" color='#780404'><b> Exploring the merged dataset </b></font>  

In [16]:
cpi_pivot.head()

,Month,Alcoholic beverages and tobacco,Clothing and footwear,Communication,Education,Food and non-alcoholic beverages,Headline_CPI,Health,Household contents and services,Housing and utilities,Miscellaneous goods and services,Recreation and culture,Restaurants and hotels,Transport,year_month,Total_Local Sales,Total_Export_Sales
0,2022-01-31,100.0,100.2,99.8,100.0,100.9,100.2,100.1,100.4,100.0,100.6,100.2,101.2,98.9,2022-01,41382.0,19089.0
1,2022-02-28,100.6,100.3,99.7,100.0,101.8,100.8,103.2,100.4,100.1,101.5,100.3,101.9,100.6,2022-02,44229.0,32867.0
2,2022-03-31,102.8,100.5,99.7,104.4,102.4,101.8,103.7,100.8,100.5,101.9,100.4,103.0,103.7,2022-03,50607.0,34285.0
3,2022-04-30,103.6,100.7,99.6,104.4,103.1,102.4,104.0,101.1,100.5,102.8,100.7,103.3,105.1,2022-04,37107.0,30788.0
4,2022-05-31,104.4,100.9,99.9,104.4,105.3,103.1,104.3,101.8,100.6,102.9,101.0,103.8,105.9,2022-05,39177.0,25786.0


In [17]:
cpi_pivot.shape

(16, 17)

- we have now added three new columns from the `vehicles` dataset, hence the number of columns have now increased to 17.


<a id='feature-correlation'></a>
<font size="+6" color='#780404'><b> Feature Engineering </b></font>  

In [18]:
feats_to_lag = [col for col in cpi_pivot.columns if col not in ['Month', 'year_month']]
for col in feats_to_lag:
  for i in range(1, 8):
    cpi_pivot[f"prev_{i}_month_{col}"]= cpi_pivot[col].shift(i)

cpi_pivot

/var/folders/mc/wtr2th093fscq0l5wyyg1yg00000gq/T/ipykernel_38473/268584411.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cpi_pivot[f"prev_{i}_month_{col}"]= cpi_pivot[col].shift(i)
/var/folders/mc/wtr2th093fscq0l5wyyg1yg00000gq/T/ipykernel_38473/268584411.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cpi_pivot[f"prev_{i}_month_{col}"]= cpi_pivot[col].shift(i)
/var/folders/mc/wtr2th093fscq0l5wyyg1yg00000gq/T/ipykernel_38473/268584411.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result

,Month,Alcoholic beverages and tobacco,Clothing and footwear,Communication,Education,Food and non-alcoholic beverages,Headline_CPI,Health,Household contents and services,Housing and utilities,Miscellaneous goods and services,Recreation and culture,Restaurants and hotels,Transport,year_month,Total_Local Sales,Total_Export_Sales,prev_1_month_Alcoholic beverages and tobacco,prev_2_month_Alcoholic beverages and tobacco,prev_3_month_Alcoholic beverages and tobacco,prev_4_month_Alcoholic beverages and tobacco,prev_5_month_Alcoholic beverages and tobacco,prev_6_month_Alcoholic beverages and tobacco,prev_7_month_Alcoholic beverages and tobacco,prev_1_month_Clothing and footwear,prev_2_month_Clothing and footwear,prev_3_month_Clothing and footwear,prev_4_month_Clothing and footwear,prev_5_month_Clothing and footwear,prev_6_month_Clothing and footwear,prev_7_month_Clothing and footwear,prev_1_month_Communication,prev_2_month_Communication,prev_3_month_Communication,prev_4_month_Communication,prev_5_month_Communication,prev_6_month_Communication,prev_7_month_Communication,prev_1_month_Education,prev_2_month_Education,prev_3_month_Education,prev_4_month_Education,prev_5_month_Education,prev_6_month_Education,prev_7_month_Education,prev_1_month_Food and non-alcoholic beverages,prev_2_month_Food and non-alcoholic beverages,prev_3_month_Food and non-alcoholic beverages,prev_4_month_Food and non-alcoholic beverages,prev_5_month_Food and non-alcoholic beverages,prev_6_month_Food and non-alcoholic beverages,prev_7_month_Food and non-alcoholic beverages,prev_1_month_Headline_CPI,prev_2_month_Headline_CPI,prev_3_month_Headline_CPI,prev_4_month_Headline_CPI,prev_5_month_Headline_CPI,prev_6_month_Headline_CPI,prev_7_month_Headline_CPI,prev_1_month_Health,prev_2_month_Health,prev_3_month_Health,prev_4_month_Health,prev_5_month_Health,prev_6_month_Health,prev_7_month_Health,prev_1_month_Household contents and services,prev_2_month_Household contents and services,prev_3_month_Household contents and services,prev_4_month_Household contents and services,prev_5_month_Household contents and services,prev_6_month_Household contents and services,prev_7_month_Household contents and services,prev_1_month_Housing and utilities,prev_2_month_Housing and utilities,prev_3_month_Housing and utilities,prev_4_month_Housing and utilities,prev_5_month_Housing and utilities,prev_6_month_Housing and utilities,prev_7_month_Housing and utilities,prev_1_month_Miscellaneous goods and services,prev_2_month_Miscellaneous goods and services,prev_3_month_Miscellaneous goods and services,prev_4_month_Miscellaneous goods and services,prev_5_month_Miscellaneous goods and services,prev_6_month_Miscellaneous goods and services,prev_7_month_Miscellaneous goods and services,prev_1_month_Recreation and culture,prev_2_month_Recreation and culture,prev_3_month_Recreation and culture,prev_4_month_Recreation and culture,prev_5_month_Recreation and culture,prev_6_month_Recreation and culture,prev_7_month_Recreation and culture,prev_1_month_Restaurants and hotels,prev_2_month_Restaurants and hotels,prev_3_month_Restaurants and hotels,prev_4_month_Restaurants and hotels,prev_5_month_Restaurants and hotels,prev_6_month_Restaurants and hotels,prev_7_month_Restaurants and hotels,prev_1_month_Transport,prev_2_month_Transport,prev_3_month_Transport,prev_4_month_Transport,prev_5_month_Transport,prev_6_month_Transport,prev_7_month_Transport,prev_1_month_Total_Local Sales,prev_2_month_Total_Local Sales,prev_3_month_Total_Local Sales,prev_4_month_Total_Local Sales,prev_5_month_Total_Local Sales,prev_6_month_Total_Local Sales,prev_7_month_Total_Local Sales,prev_1_month_Total_Export_Sales,prev_2_month_Total_Export_Sales,prev_3_month_Total_Export_Sales,prev_4_month_Total_Export_Sales,prev_5_month_Total_Export_Sales,prev_6_month_Total_Export_Sales,prev_7_month_Total_Export_Sales
0,2022-01-31,100.0,100.2,99.8,100.0,100.9,100.2,100.1,100.4,100.0,100.6,100.2,101.2,98.9,2022-01,41382.0,19089.0,NaN,NaN,NaN,NaN,

In [19]:
cpi_pivot = cpi_pivot.drop(0)
cpi_pivot = cpi_pivot.bfill()
cpi_pivot

,Month,Alcoholic beverages and tobacco,Clothing and footwear,Communication,Education,Food and non-alcoholic beverages,Headline_CPI,Health,Household contents and services,Housing and utilities,Miscellaneous goods and services,Recreation and culture,Restaurants and hotels,Transport,year_month,Total_Local Sales,Total_Export_Sales,prev_1_month_Alcoholic beverages and tobacco,prev_2_month_Alcoholic beverages and tobacco,prev_3_month_Alcoholic beverages and tobacco,prev_4_month_Alcoholic beverages and tobacco,prev_5_month_Alcoholic beverages and tobacco,prev_6_month_Alcoholic beverages and tobacco,prev_7_month_Alcoholic beverages and tobacco,prev_1_month_Clothing and footwear,prev_2_month_Clothing and footwear,prev_3_month_Clothing and footwear,prev_4_month_Clothing and footwear,prev_5_month_Clothing and footwear,prev_6_month_Clothing and footwear,prev_7_month_Clothing and footwear,prev_1_month_Communication,prev_2_month_Communication,prev_3_month_Communication,prev_4_month_Communication,prev_5_month_Communication,prev_6_month_Communication,prev_7_month_Communication,prev_1_month_Education,prev_2_month_Education,prev_3_month_Education,prev_4_month_Education,prev_5_month_Education,prev_6_month_Education,prev_7_month_Education,prev_1_month_Food and non-alcoholic beverages,prev_2_month_Food and non-alcoholic beverages,prev_3_month_Food and non-alcoholic beverages,prev_4_month_Food and non-alcoholic beverages,prev_5_month_Food and non-alcoholic beverages,prev_6_month_Food and non-alcoholic beverages,prev_7_month_Food and non-alcoholic beverages,prev_1_month_Headline_CPI,prev_2_month_Headline_CPI,prev_3_month_Headline_CPI,prev_4_month_Headline_CPI,prev_5_month_Headline_CPI,prev_6_month_Headline_CPI,prev_7_month_Headline_CPI,prev_1_month_Health,prev_2_month_Health,prev_3_month_Health,prev_4_month_Health,prev_5_month_Health,prev_6_month_Health,prev_7_month_Health,prev_1_month_Household contents and services,prev_2_month_Household contents and services,prev_3_month_Household contents and services,prev_4_month_Household contents and services,prev_5_month_Household contents and services,prev_6_month_Household contents and services,prev_7_month_Household contents and services,prev_1_month_Housing and utilities,prev_2_month_Housing and utilities,prev_3_month_Housing and utilities,prev_4_month_Housing and utilities,prev_5_month_Housing and utilities,prev_6_month_Housing and utilities,prev_7_month_Housing and utilities,prev_1_month_Miscellaneous goods and services,prev_2_month_Miscellaneous goods and services,prev_3_month_Miscellaneous goods and services,prev_4_month_Miscellaneous goods and services,prev_5_month_Miscellaneous goods and services,prev_6_month_Miscellaneous goods and services,prev_7_month_Miscellaneous goods and services,prev_1_month_Recreation and culture,prev_2_month_Recreation and culture,prev_3_month_Recreation and culture,prev_4_month_Recreation and culture,prev_5_month_Recreation and culture,prev_6_month_Recreation and culture,prev_7_month_Recreation and culture,prev_1_month_Restaurants and hotels,prev_2_month_Restaurants and hotels,prev_3_month_Restaurants and hotels,prev_4_month_Restaurants and hotels,prev_5_month_Restaurants and hotels,prev_6_month_Restaurants and hotels,prev_7_month_Restaurants and hotels,prev_1_month_Transport,prev_2_month_Transport,prev_3_month_Transport,prev_4_month_Transport,prev_5_month_Transport,prev_6_month_Transport,prev_7_month_Transport,prev_1_month_Total_Local Sales,prev_2_month_Total_Local Sales,prev_3_month_Total_Local Sales,prev_4_month_Total_Local Sales,prev_5_month_Total_Local Sales,prev_6_month_Total_Local Sales,prev_7_month_Total_Local Sales,prev_1_month_Total_Export_Sales,prev_2_month_Total_Export_Sales,prev_3_month_Total_Export_Sales,prev_4_month_Total_Export_Sales,prev_5_month_Total_Export_Sales,prev_6_month_Total_Export_Sales,prev_7_month_Total_Export_Sales
1,2022-02-28,100.6,100.3,99.7,100.0,101.8,100.8,103.2,100.4,100.1,101.5,100.3,101.9,100.6,2022-02,44229.0,32867.0,100.0,100.0,100

In [20]:
cpi_pivot.shape

(15, 122)

In [21]:
train = cpi_pivot[cpi_pivot['Month'] != '2023-04-30']
test = cpi_pivot[cpi_pivot['Month'] == '2023-04-30']

training_set = train[train['Month']!= '2023-03-31']
validation_set = train[train['Month']== '2023-03-31']

train.shape, test.shape, training_set.shape, validation_set.shape

((14, 122), (1, 122), (13, 122), (1, 122))

In [22]:
from sklearn.preprocessing import MinMaxScaler


In [23]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

<a id='Model Selection'></a>
<font size="+3" color='#780404'><b> Model Selection: Advanced Regression Models</b></font>  
<a id='modeling'></a>

In [24]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor

# Create a list of regression models with MinMaxScaler
model_pipeline = []

# Linear Regression with MinMaxScaler
model_pipeline.append(Pipeline([
    ('scaler', MinMaxScaler()),  # Use MinMaxScaler
    ('regressor', LinearRegression())
]))

# Random Forest Regressor
model_pipeline.append(RandomForestRegressor(n_estimators=100, random_state=42))

# Decision Tree Regressor
model_pipeline.append(DecisionTreeRegressor())

# Support Vector Regressor
model_pipeline.append(SVR())

# Ridge Regression with MinMaxScaler
model_pipeline.append(Pipeline([
    ('scaler', MinMaxScaler()),  # Use MinMaxScaler
    ('regressor', Ridge())
]))

# K-Nearest Neighbors Regressor
model_pipeline.append(KNeighborsRegressor())

# Gradient Boosting Regressor
model_pipeline.append(GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42))


In [25]:
target_cols = ['Alcoholic beverages and tobacco', 'Clothing and footwear',
       'Communication', 'Education', 'Food and non-alcoholic beverages',
       'Headline_CPI', 'Health', 'Household contents and services',
       'Housing and utilities', 'Miscellaneous goods and services',
       'Recreation and culture', 'Restaurants and hotels ', 'Transport']

#if you add additional data sources that have no value in the predicting month , drop it, now that you have their lags
features= [col for col in train.columns if col not in target_cols + ['Month', 'year_month','Total_Local Sales', 'Total_Export_Sales']]


X_train = training_set[features]
y_train = training_set[target_cols]

X_val = validation_set[features]
y_val = validation_set[target_cols]


lr_models = {}
y_pred = []
scaler = MinMaxScaler()

#training
for target_col in target_cols:
  lr_model = LinearRegression()
  X_train_scaled = scaler.fit_transform(X_train)
  lr_model.fit(X_train_scaled, y_train[target_col])
  lr_models[target_col] = lr_model

#validation

for target_col in target_cols:
  lr_model = lr_models[target_col]
  X_val_scaled = scaler.transform(X_val)
  y_pred_col = lr_model.predict(X_val_scaled)
  y_pred.append(y_pred_col)


#scoring

y_pred = np.array(y_pred).T

df = pd.DataFrame({'y_pred': y_pred.flatten(), 'y_val': y_val.values.flatten()})

#calculate the rmse
rmse = np.sqrt(mean_squared_error(df['y_pred'], df['y_val']))

print(f'RMSE: {rmse}')







RMSE: 0.7170057743628517


<a id='feature-correlation'></a>
<font size="+6" color='#780404'><b> Making Predictions for April 2023 </b></font>  

In [26]:
X_train.head()

,prev_1_month_Alcoholic beverages and tobacco,prev_2_month_Alcoholic beverages and tobacco,prev_3_month_Alcoholic beverages and tobacco,prev_4_month_Alcoholic beverages and tobacco,prev_5_month_Alcoholic beverages and tobacco,prev_6_month_Alcoholic beverages and tobacco,prev_7_month_Alcoholic beverages and tobacco,prev_1_month_Clothing and footwear,prev_2_month_Clothing and footwear,prev_3_month_Clothing and footwear,prev_4_month_Clothing and footwear,prev_5_month_Clothing and footwear,prev_6_month_Clothing and footwear,prev_7_month_Clothing and footwear,prev_1_month_Communication,prev_2_month_Communication,prev_3_month_Communication,prev_4_month_Communication,prev_5_month_Communication,prev_6_month_Communication,prev_7_month_Communication,prev_1_month_Education,prev_2_month_Education,prev_3_month_Education,prev_4_month_Education,prev_5_month_Education,prev_6_month_Education,prev_7_month_Education,prev_1_month_Food and non-alcoholic beverages,prev_2_month_Food and non-alcoholic beverages,prev_3_month_Food and non-alcoholic beverages,prev_4_month_Food and non-alcoholic beverages,prev_5_month_Food and non-alcoholic beverages,prev_6_month_Food and non-alcoholic beverages,prev_7_month_Food and non-alcoholic beverages,prev_1_month_Headline_CPI,prev_2_month_Headline_CPI,prev_3_month_Headline_CPI,prev_4_month_Headline_CPI,prev_5_month_Headline_CPI,prev_6_month_Headline_CPI,prev_7_month_Headline_CPI,prev_1_month_Health,prev_2_month_Health,prev_3_month_Health,prev_4_month_Health,prev_5_month_Health,prev_6_month_Health,prev_7_month_Health,prev_1_month_Household contents and services,prev_2_month_Household contents and services,prev_3_month_Household contents and services,prev_4_month_Household contents and services,prev_5_month_Household contents and services,prev_6_month_Household contents and services,prev_7_month_Household contents and services,prev_1_month_Housing and utilities,prev_2_month_Housing and utilities,prev_3_month_Housing and utilities,prev_4_month_Housing and utilities,prev_5_month_Housing and utilities,prev_6_month_Housing and utilities,prev_7_month_Housing and utilities,prev_1_month_Miscellaneous goods and services,prev_2_month_Miscellaneous goods and services,prev_3_month_Miscellaneous goods and services,prev_4_month_Miscellaneous goods and services,prev_5_month_Miscellaneous goods and services,prev_6_month_Miscellaneous goods and services,prev_7_month_Miscellaneous goods and services,prev_1_month_Recreation and culture,prev_2_month_Recreation and culture,prev_3_month_Recreation and culture,prev_4_month_Recreation and culture,prev_5_month_Recreation and culture,prev_6_month_Recreation and culture,prev_7_month_Recreation and culture,prev_1_month_Restaurants and hotels,prev_2_month_Restaurants and hotels,prev_3_month_Restaurants and hotels,prev_4_month_Restaurants and hotels,prev_5_month_Restaurants and hotels,prev_6_month_Restaurants and hotels,prev_7_month_Restaurants and hotels,prev_1_month_Transport,prev_2_month_Transport,prev_3_month_Transport,prev_4_month_Transport,prev_5_month_Transport,prev_6_month_Transport,prev_7_month_Transport,prev_1_month_Total_Local Sales,prev_2_month_Total_Local Sales,prev_3_month_Total_Local Sales,prev_4_month_Total_Local Sales,prev_5_month_Total_Local Sales,prev_6_month_Total_Local Sales,prev_7_month_Total_Local Sales,prev_1_month_Total_Export_Sales,prev_2_month_Total_Export_Sales,prev_3_month_Total_Export_Sales,prev_4_month_Total_Export_Sales,prev_5_month_Total_Export_Sales,prev_6_month_Total_Export_Sales,prev_7_month_Total_Export_Sales
1,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.2,100.2,100.2,100.2,100.2,100.2,100.2,99.8,99.8,99.8,99.8,99.8,99.8,99.8,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.9,100.9,100.9,100.9,100.9,100.9,100.9,100.2,100.2,100.2,100.2,100.2,100.2,100.2,100.1,100.1,100.1,100.1,100.1,100.1,100.1,100.4,100.4,100.4,100.4,100.4,100.4,100.4,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.6,100.6,100.6,100.6,100.6,100.6,100.6,100.2,100.2,100.2,100.2,100.2,100.2,100.2,

In [27]:
first_row = X_train.copy()

# Method 2: Create a new DataFrame 'first_row' using slicing
first_row = X_train.iloc[[1]].copy()

# Note: In Method 2, we are extracting the first row (index 0) from the DataFrame 'X'
# and creating a new DataFrame 'first_row' using the `copy()` method to avoid modifying the original data.

# Display the 'first_row' DataFrame
first_row

,prev_1_month_Alcoholic beverages and tobacco,prev_2_month_Alcoholic beverages and tobacco,prev_3_month_Alcoholic beverages and tobacco,prev_4_month_Alcoholic beverages and tobacco,prev_5_month_Alcoholic beverages and tobacco,prev_6_month_Alcoholic beverages and tobacco,prev_7_month_Alcoholic beverages and tobacco,prev_1_month_Clothing and footwear,prev_2_month_Clothing and footwear,prev_3_month_Clothing and footwear,prev_4_month_Clothing and footwear,prev_5_month_Clothing and footwear,prev_6_month_Clothing and footwear,prev_7_month_Clothing and footwear,prev_1_month_Communication,prev_2_month_Communication,prev_3_month_Communication,prev_4_month_Communication,prev_5_month_Communication,prev_6_month_Communication,prev_7_month_Communication,prev_1_month_Education,prev_2_month_Education,prev_3_month_Education,prev_4_month_Education,prev_5_month_Education,prev_6_month_Education,prev_7_month_Education,prev_1_month_Food and non-alcoholic beverages,prev_2_month_Food and non-alcoholic beverages,prev_3_month_Food and non-alcoholic beverages,prev_4_month_Food and non-alcoholic beverages,prev_5_month_Food and non-alcoholic beverages,prev_6_month_Food and non-alcoholic beverages,prev_7_month_Food and non-alcoholic beverages,prev_1_month_Headline_CPI,prev_2_month_Headline_CPI,prev_3_month_Headline_CPI,prev_4_month_Headline_CPI,prev_5_month_Headline_CPI,prev_6_month_Headline_CPI,prev_7_month_Headline_CPI,prev_1_month_Health,prev_2_month_Health,prev_3_month_Health,prev_4_month_Health,prev_5_month_Health,prev_6_month_Health,prev_7_month_Health,prev_1_month_Household contents and services,prev_2_month_Household contents and services,prev_3_month_Household contents and services,prev_4_month_Household contents and services,prev_5_month_Household contents and services,prev_6_month_Household contents and services,prev_7_month_Household contents and services,prev_1_month_Housing and utilities,prev_2_month_Housing and utilities,prev_3_month_Housing and utilities,prev_4_month_Housing and utilities,prev_5_month_Housing and utilities,prev_6_month_Housing and utilities,prev_7_month_Housing and utilities,prev_1_month_Miscellaneous goods and services,prev_2_month_Miscellaneous goods and services,prev_3_month_Miscellaneous goods and services,prev_4_month_Miscellaneous goods and services,prev_5_month_Miscellaneous goods and services,prev_6_month_Miscellaneous goods and services,prev_7_month_Miscellaneous goods and services,prev_1_month_Recreation and culture,prev_2_month_Recreation and culture,prev_3_month_Recreation and culture,prev_4_month_Recreation and culture,prev_5_month_Recreation and culture,prev_6_month_Recreation and culture,prev_7_month_Recreation and culture,prev_1_month_Restaurants and hotels,prev_2_month_Restaurants and hotels,prev_3_month_Restaurants and hotels,prev_4_month_Restaurants and hotels,prev_5_month_Restaurants and hotels,prev_6_month_Restaurants and hotels,prev_7_month_Restaurants and hotels,prev_1_month_Transport,prev_2_month_Transport,prev_3_month_Transport,prev_4_month_Transport,prev_5_month_Transport,prev_6_month_Transport,prev_7_month_Transport,prev_1_month_Total_Local Sales,prev_2_month_Total_Local Sales,prev_3_month_Total_Local Sales,prev_4_month_Total_Local Sales,prev_5_month_Total_Local Sales,prev_6_month_Total_Local Sales,prev_7_month_Total_Local Sales,prev_1_month_Total_Export_Sales,prev_2_month_Total_Export_Sales,prev_3_month_Total_Export_Sales,prev_4_month_Total_Export_Sales,prev_5_month_Total_Export_Sales,prev_6_month_Total_Export_Sales,prev_7_month_Total_Export_Sales
2,100.6,100.0,100.0,100.0,100.0,100.0,100.0,100.3,100.2,100.2,100.2,100.2,100.2,100.2,99.7,99.8,99.8,99.8,99.8,99.8,99.8,100.0,100.0,100.0,100.0,100.0,100.0,100.0,101.8,100.9,100.9,100.9,100.9,100.9,100.9,100.8,100.2,100.2,100.2,100.2,100.2,100.2,103.2,100.1,100.1,100.1,100.1,100.1,100.1,100.4,100.4,100.4,100.4,100.4,100.4,100.4,100.1,100.0,100.0,100.0,100.0,100.0,100.0,101.5,100.6,100.6,100.6,100.6,100.6,100.6,100.3,100.2,100.2,100.2,100.2,100.2,100.2,

In [28]:
y_pred_test = []

for target_col in target_cols:
    lr_model = lr_models[target_col]
    X_test_scaled = scaler.transform(X_val)  # Scale the test data
    y_pred_col = lr_model.predict(X_test_scaled)  # Make predictions
    y_pred_test.append(y_pred_col)

# Combine predictions into a DataFrame
df_pred = pd.DataFrame({col: y_pred_test[i] for i, col in enumerate(target_cols)})


In [29]:
df_pred 

,Alcoholic beverages and tobacco,Clothing and footwear,Communication,Education,Food and non-alcoholic beverages,Headline_CPI,Health,Household contents and services,Housing and utilities,Miscellaneous goods and services,Recreation and culture,Restaurants and hotels,Transport
0,108.736709,103.708518,99.561444,108.35541,116.626424,108.998903,108.121907,107.32011,104.987787,107.77208,103.724011,109.327778,113.55311


In [30]:
test.head()

,Month,Alcoholic beverages and tobacco,Clothing and footwear,Communication,Education,Food and non-alcoholic beverages,Headline_CPI,Health,Household contents and services,Housing and utilities,Miscellaneous goods and services,Recreation and culture,Restaurants and hotels,Transport,year_month,Total_Local Sales,Total_Export_Sales,prev_1_month_Alcoholic beverages and tobacco,prev_2_month_Alcoholic beverages and tobacco,prev_3_month_Alcoholic beverages and tobacco,prev_4_month_Alcoholic beverages and tobacco,prev_5_month_Alcoholic beverages and tobacco,prev_6_month_Alcoholic beverages and tobacco,prev_7_month_Alcoholic beverages and tobacco,prev_1_month_Clothing and footwear,prev_2_month_Clothing and footwear,prev_3_month_Clothing and footwear,prev_4_month_Clothing and footwear,prev_5_month_Clothing and footwear,prev_6_month_Clothing and footwear,prev_7_month_Clothing and footwear,prev_1_month_Communication,prev_2_month_Communication,prev_3_month_Communication,prev_4_month_Communication,prev_5_month_Communication,prev_6_month_Communication,prev_7_month_Communication,prev_1_month_Education,prev_2_month_Education,prev_3_month_Education,prev_4_month_Education,prev_5_month_Education,prev_6_month_Education,prev_7_month_Education,prev_1_month_Food and non-alcoholic beverages,prev_2_month_Food and non-alcoholic beverages,prev_3_month_Food and non-alcoholic beverages,prev_4_month_Food and non-alcoholic beverages,prev_5_month_Food and non-alcoholic beverages,prev_6_month_Food and non-alcoholic beverages,prev_7_month_Food and non-alcoholic beverages,prev_1_month_Headline_CPI,prev_2_month_Headline_CPI,prev_3_month_Headline_CPI,prev_4_month_Headline_CPI,prev_5_month_Headline_CPI,prev_6_month_Headline_CPI,prev_7_month_Headline_CPI,prev_1_month_Health,prev_2_month_Health,prev_3_month_Health,prev_4_month_Health,prev_5_month_Health,prev_6_month_Health,prev_7_month_Health,prev_1_month_Household contents and services,prev_2_month_Household contents and services,prev_3_month_Household contents and services,prev_4_month_Household contents and services,prev_5_month_Household contents and services,prev_6_month_Household contents and services,prev_7_month_Household contents and services,prev_1_month_Housing and utilities,prev_2_month_Housing and utilities,prev_3_month_Housing and utilities,prev_4_month_Housing and utilities,prev_5_month_Housing and utilities,prev_6_month_Housing and utilities,prev_7_month_Housing and utilities,prev_1_month_Miscellaneous goods and services,prev_2_month_Miscellaneous goods and services,prev_3_month_Miscellaneous goods and services,prev_4_month_Miscellaneous goods and services,prev_5_month_Miscellaneous goods and services,prev_6_month_Miscellaneous goods and services,prev_7_month_Miscellaneous goods and services,prev_1_month_Recreation and culture,prev_2_month_Recreation and culture,prev_3_month_Recreation and culture,prev_4_month_Recreation and culture,prev_5_month_Recreation and culture,prev_6_month_Recreation and culture,prev_7_month_Recreation and culture,prev_1_month_Restaurants and hotels,prev_2_month_Restaurants and hotels,prev_3_month_Restaurants and hotels,prev_4_month_Restaurants and hotels,prev_5_month_Restaurants and hotels,prev_6_month_Restaurants and hotels,prev_7_month_Restaurants and hotels,prev_1_month_Transport,prev_2_month_Transport,prev_3_month_Transport,prev_4_month_Transport,prev_5_month_Transport,prev_6_month_Transport,prev_7_month_Transport,prev_1_month_Total_Local Sales,prev_2_month_Total_Local Sales,prev_3_month_Total_Local Sales,prev_4_month_Total_Local Sales,prev_5_month_Total_Local Sales,prev_6_month_Total_Local Sales,prev_7_month_Total_Local Sales,prev_1_month_Total_Export_Sales,prev_2_month_Total_Export_Sales,prev_3_month_Total_Export_Sales,prev_4_month_Total_Export_Sales,prev_5_month_Total_Export_Sales,prev_6_month_Total_Export_Sales,prev_7_month_Total_Export_Sales
15,2023-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04,NaN,NaN,109.2,106.9,106.5,106.2,106.4,106.1,105.6,103.4

In [31]:
import joblib

for target_col in target_cols:
    lr_model = lr_models[target_col]
    joblib.dump(lr_model, f"{target_col}_model.pkl")